# Logistic Regression

In [1]:
import math
import numpy as np
import pandas as pd
import sklearn.linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt

%matplotlib inline

## Simulierte Daten

In [2]:
observations_count = 1000
def generate_prices(number):
    return [np.round(np.random.uniform(1,20), decimals=1) for i in range(0, number)]

prices = generate_prices(observations_count)
competitors_count = 5
competitor_prices = np.matrix([generate_prices(observations_count) for i in range(0, competitors_count)])
ranks = [1 + len([1 for j in range(0, competitors_count) if competitor_prices[j,i] < prices[i]]) 
         for i in range(0, observations_count)]
sold = [np.maximum(0, np.round(np.random.uniform(0, 1-0.3*ranks[i]/(competitors_count+1)-0.05*prices[i]))) 
        for i in range(0, observations_count)]

## Regression

In [3]:
explanatory_count = 3
data = []
for i in range(0, explanatory_count):
    if i == 0:
        data.append([1 for j in range(0, observations_count)])
    elif i == 1:
        data.append([ranks[j] for j in range(0, observations_count)])
    else:
        data.append([prices[j] - np.min([competitor_prices[k,j] for k in range(0, competitors_count)]) for j in range(0, observations_count)])
explanatory = np.matrix(data)
logit = sm.Logit(sold, explanatory.transpose())
result = logit.fit()
beta = result.params
beta

Optimization terminated successfully.
         Current function value: 0.229029
         Iterations 9


array([ 0.23952785, -0.84845228, -0.13670752])

In [4]:
result.aic

464.05813712380848

In [5]:
predicted_solds = [np.exp(np.sum([beta[m]*explanatory[m,i] for m in range(0, explanatory_count)])) / (1 + np.exp(np.sum([beta[m]*explanatory[m,i] for m in range(0, explanatory_count)]))) for i in range(0, observations_count)]

## Optimierung

In [7]:
beta_demand = beta
competitor_prices = generate_prices(competitors_count)
prices_range = np.arange(1, 20.1, 0.1)
data = []
for i in range(explanatory_count):
    if i == 0:
        data.append([1 for price in prices_range])
    elif i == 1:
        data.append([1 + len([1 for j in range(0, competitors_count) if competitor_prices[j] < price]) for price in prices_range])
    else:
        data.append([price - np.min([competitor_prices[j] for j in range(competitors_count)]) for price in prices_range])
explanatory = np.matrix(data)

data = []
for i in range(2):
    p = [np.exp(np.sum([beta_demand[m]*explanatory[m,price] for m in range(explanatory_count)])) / (1 + np.exp(np.sum([beta_demand[m]*explanatory[m,price] for m in range(explanatory_count)]))) for price in prices_range]
    if i == 1:
        data.append(p)
    else:
        data.append([1-j for j in p])
sale_prob = np.matrix(data)

delta = 0.99
holding_cost_rate = 0.01
init_inventory = 15
steps = 100
values = np.empty(shape=(steps+1, init_inventory))
for step in range(steps, -1, -1):
    for n in range(init_inventory):
        if step == steps or n == 0:
            values[step, n] = 0
        else:
            values[step, n] = np.max([np.sum([sale_prob[i,price]*(np.min([n,i]) * price - n * holding_cost_rate + delta * values[step+1, np.max([0, n-i])]) for i in range(2)]) for price in prices_range])

opt_prices = np.empty(init_inventory)
for n in range(init_inventory):
    for price in prices_range:
        if np.sum([sale_prob[i,price]*(np.min([n,i]) * price - n * holding_cost_rate + delta * values[1, np.max([0, n-i])]) for i in range(2)]) == values[0, n]:
            opt_prices[n] = price
opt_values = [values[0,n] for n in range(init_inventory)]
opt_prices

array([ 20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,
        20.,  20.,  20.,  20.])